In [1]:
import sympy as sp
import numpy as np
import pandas as pd
import time
import warnings
import torch
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [2]:
def funF(u1, u2, A):
  return np.log((u1 + (np.sqrt((u1)**2 +(A)**2)))/(u2 + (np.sqrt((u2)**2+(A)**2))))

In [3]:
def funAz(x, y, x_est, y_est):
  return np.sqrt((x - x_est)**2 + (y - y_est)**2)

In [4]:
def FunPosicao1(tamanho_haste, passo, distancia):
    eixo_x = np.arange(-distancia, distancia+passo, passo)
    eixo_y = np.arange(-distancia, distancia+passo, passo)
    eixo_z = np.linspace(-(2 * tamanho_haste), 2 * tamanho_haste, len(eixo_x))
    # eixo_z = np.arange(-(2 * tamanho_haste + 1), 2 * tamanho_haste + 1 + passo, len(eixo_x))
    return eixo_x, eixo_y, eixo_z

In [5]:
def calculo_potencial(x,y,z,num_hastes,Profundidade_Injetado_Haste,reflex,K,I,posi_haste_x,posi_haste_y,profundidade_1_camada,Rho_solo1,h):
  tensao_gerada = np.zeros((len(x), len(y), len(z), num_hastes))
  Az = np.zeros((len(x), len(y), len(z), num_hastes))
  f1 = np.zeros((len(x), len(y), len(z), num_hastes))
  f2 = np.zeros((len(x), len(y), len(z), num_hastes))
  f3 = np.zeros((len(x), len(y), len(z), num_hastes))
  f4 = np.zeros((len(x), len(y), len(z), num_hastes))
  for i in range(0, len(x)):
    for j in range(0, len(y)):
      for k in range(0, len(z)):
        for l in range(0, num_hastes):
          Az[i,j,k,l] = funAz(x[i], y[j], posi_haste_x[l], posi_haste_y[l]);
          f1[i,j,k,l] = funF(profundidade_1_camada-z[k], h-z[k], Az[i,j,k,l]);
          f2[i,j,k,l] = funF(profundidade_1_camada+z[k], h+z[k], Az[i,j,k,l]);
          f3[i,j,k,l] = funF(h+Profundidade_Injetado_Haste-z[k], profundidade_1_camada-z[k], Az[i,j,k,l]);
          f4[i,j,k,l] = funF(h+Profundidade_Injetado_Haste+z[k], profundidade_1_camada+z[k], Az[i,j,k,l]);
          soma1 = 0
          soma2 = 0
          for cont in range(0, reflex+1):
            f5 = funF(2*cont*profundidade_1_camada+profundidade_1_camada-z[k], 2*cont*profundidade_1_camada+h-z[k], Az[i,j,k,l])
            f6 = funF(2*cont*profundidade_1_camada+profundidade_1_camada+z[k], 2*cont*profundidade_1_camada+h+z[k], Az[i,j,k,l])
            f7 = funF(2*cont*profundidade_1_camada-h-z[k], 2*cont*profundidade_1_camada-profundidade_1_camada-z[k], Az[i,j,k,l])
            f8 = funF(2*cont*profundidade_1_camada-h+z[k], 2*cont*profundidade_1_camada-profundidade_1_camada+z[k], Az[i,j,k,l])
            f9 = funF(2*cont*profundidade_1_camada+h+Profundidade_Injetado_Haste-z[k], 2*cont*profundidade_1_camada+profundidade_1_camada-z[k], Az)
            f10 = funF(2*cont*profundidade_1_camada+h+Profundidade_Injetado_Haste+z[k], 2*cont*profundidade_1_camada+profundidade_1_camada+z[k], Az)
            soma1=soma1+K**cont*(f5+f6+f7+f8)
            soma2=soma2+K**cont*(f9+f10)
          tensao_gerada[i,j,k,l]=1/(4*np.pi*Profundidade_Injetado_Haste)*Rho_solo1*I[l]*(f1[i,j,k,l]+f2[i,j,k,l]+(1+K)*(f3[i,j,k,l]+f4[i,j,k,l])+soma1+(1+K)*(soma2))
  return tensao_gerada

In [6]:
def calculo_superficial(x, y, z, Vc, n):
  Vsup = np.zeros((len(x), len(y)))
  V = np.zeros((len(x), len(y), len(z)))
  for i in range(0, len(x)):
      for j in range(0, len(y)):
          for k in range(0, len(z)):
              for o in range(0, n):
                  V[i, j, k] = V[i, j, k] + Vc[i, j, k, o]
  for cont1 in range(0, len(x)):
      for cont2 in range(0, len(y)):
          Vsup[cont1, cont2] = V[cont1, cont2, (len(z) // 2) - 1]
  return Vsup

In [7]:
def modelagem_duas_camadas(rho_1_camada, profundidade_1_camada, rho_2_camada, num_hastes, tamanho_haste, reflex, h, passo, tamanho_maximo, posi_haste_x, posi_haste_y, corrente_haste):
    # posi_haste_x = np.ndarray(num_hastes)
    # posi_haste_y = np.ndarray(num_hastes)
    # corrente_haste = np.ndarray(num_hastes)
    # for temp_haste in range(0, num_hastes):
    #         print(f'Digite a posição da haste {temp_haste + 1} no eixo X em "m":')
    #         posi_haste_x[temp_haste] = float(input())
    #         print(f'Digite a posição da haste {temp_haste + 1} no eixo Y em "m":')
    #         posi_haste_y[temp_haste] = float(input())
    #         print(f'Digite a corrente da haste {temp_haste + 1}: ')
    #         corrente_haste[temp_haste] = float(input())
    x, y, z = FunPosicao1(tamanho_haste, passo, tamanho_maximo)
    K = (rho_2_camada - rho_1_camada) / (rho_2_camada + rho_1_camada)
    tensao_solo = calculo_potencial(x, y, z, num_hastes, tamanho_haste, reflex, K, corrente_haste, posi_haste_x, posi_haste_y, profundidade_1_camada, rho_1_camada, h)
    tensao_superficial = calculo_superficial(x, y, z, tensao_solo, num_hastes)
    return tensao_superficial

In [8]:
num_hastes = 1
posi_haste_x = np.ndarray(num_hastes)
posi_haste_y = np.ndarray(num_hastes)
corrente_haste = np.ndarray(num_hastes)
posi_haste_x[0] = 0.0
posi_haste_y[0] = 0.0
corrente_haste[0] = 10.0

In [12]:
time1 = time.time()
valores1 = modelagem_duas_camadas(68.285, 1.285, 56.874, 1, 3, 50, 0, 0.1, 3, posi_haste_x, posi_haste_y, corrente_haste)
tTime1 = (time.time() - time1)
print("Tempo 1 haste: " + str(tTime1))

ValueError: setting an array element with a sequence.

In [13]:
np.savetxt('valores1.csv', valores1)

NameError: name 'valores1' is not defined

In [11]:
time2 = time.time()
valores2 = modelagem_duas_camadas(68.285, 1.285, 56.874, 1, 3, 50, 0, 0.1, 12, posi_haste_x, posi_haste_y, corrente_haste)
tTime2 = (time.time() - time2)
print("Tempo 12 Hastes: " + str(tTime2))

KeyboardInterrupt: 

In [ ]:
np.savetxt('valores2.csv', valores2)